# Домашнее задание «Сегментация и детекция объектов»

## Распознавание рукописного ввода на примере базы MNIST

Построить классификатор изображений рукописного ввода на базе MNIST. В качестве шаблона в данной работе можно использовать ipython-ноутбук 002-digit.ipynb.

Классификатор предлагается строить на признаках, полученных в результате предобработки изображений, например, гистограммы градиентов (HOG) или результат PCA преобразования.

В качестве модели классификатора можно использовать любую известную Вам модель, за исключением сверточных нейронных сетей.

Критерием качества классификатора является метрика accuracy. Для получения зачета по данной работе, значение метрики accuracy должно быть больше 0.6. Метрика оценивается на тестовой выборке в рамках контеста Digit Recognizer на Kaggle.

Решение необходимо предоставить в виде ipython-ноутбука с реализацией процесса построения модели и скриншота с финальным результатом на Kaggle.

In [5]:
# импорт библиотек
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from skimage.feature import hog
from tqdm import tqdm

# загрузка данных

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

X = train.drop("label", axis=1).values
y = train["label"].values

# разделение на обучающую и тестовую выборку
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# функция извлечения HOG-признаков
def extract_hog_features(images):
    hog_features = []
    for img in tqdm(images):
        img_reshaped = img.reshape((28, 28))
        features = hog(img_reshaped,
                       orientations=9,
                       pixels_per_cell=(8, 8),
                       cells_per_block=(2, 2),
                       block_norm='L2-Hys')
        hog_features.append(features)
    return np.array(hog_features)

# извлечение признаков
X_train_hog = extract_hog_features(X_train)
X_valid_hog = extract_hog_features(X_valid)

# стандартизация
scaler = StandardScaler()
X_train_hog = scaler.fit_transform(X_train_hog)
X_valid_hog = scaler.transform(X_valid_hog)

# обучение модели
clf = SVC(kernel='rbf', C=10, gamma=0.05)
clf.fit(X_train_hog, y_train)

# оценка точности
y_pred = clf.predict(X_valid_hog)
acc = accuracy_score(y_valid, y_pred)
print(f"Validation accuracy: {acc:.4f}")

100%|██████████| 8400/8400 [00:00<00:00, 19511.80it/s]


Validation accuracy: 0.8404


In [6]:
# извлечение признаков для тестовых данных
X_test = test.values
X_test_hog = extract_hog_features(X_test)
X_test_hog = scaler.transform(X_test_hog)

# предсказания
y_test_pred = clf.predict(X_test_hog)

# формирукм файл
submission = pd.DataFrame({
    "ImageId": np.arange(1, len(y_test_pred)+1),
    "Label": y_test_pred
})
submission.to_csv("submission.csv", index=False)
print("Готово")

100%|██████████| 28000/28000 [00:01<00:00, 18083.07it/s]


Готово
